# Model

## Import and read Data

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import os
from train_tabular import fit


os.environ["WANDB_SILENT"] = "true"

X_train = pd.read_csv(r'../../data/train_data.csv')
Y_train = X_train["Category"]

X_test = pd.read_csv(r'../../data/test_data.csv')
Y_test = X_test["Category"]
X_train= X_train.iloc[:,[i for i in range(5,12)]+ [-2] + [-1] ]#+[i for i in range(14,64)]+ [66]]
X_test= X_test.iloc[:,[i for i in range(5,12)]+ [-1]  ]#+[i for i in range(14,64)]+ [66]]

## Preprocessing

In [2]:
# define preprocessing function
#None for XGB
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.fit_transform(Y_test)

## Setup Model

In [3]:
model = XGBClassifier()

## Fit Model and Evaluate Model

In [5]:
models = []
for fold in range(0,5):
    fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,fold,"Tab-Data-XGBoost","XGBoost",None,class_names = le.fit_transform(['non-prolaktinom','prolaktinom']))
    models.append(fitted_model_fold)